<h1>Example Code Importing and Processing Data from CSV file</h1>

<h3>
    Example will go through the extraction of necessary data for the creation of an ANN extrapolating 120m wind speeds.</h3>
<h3>
    The Data.csv file contains 10 minute data. Spreadsheet foramtted to go through each individual variable at every height (40m -> 120m) before moving to the next variable. Variables include, in order:
    
    Wind Speed ('WS ##')
    Previous period wind speed ('Prev WS ##')
    Turbulence Intensity ('uTI ##')
    Vertical Wind Speed ('W ##')
    Wind Direction ('Dir ##')
    Hour of the day ('hour')
where '##' denotes data height. Hour has already been formatted as a cosine curve and direction is taken as 0º -> 360º.</h3>
<h3>
    The Angles.csv file contains elevation angles for the terrain in 30º intervals surrounding the lidar. File is made up of a single column containing twelve elevation angles.</h3>

In [ ]:
import numpy as np
import pandas as pd
import csv
import seaborn as sns

data = pd.read_csv('Data.csv', delimiter = ',')
Angles = pd.read_csv('Angles.csv', delimiter = ',')

<h2>First filter out wind speeds below 3m/s and create a Numpy array for ease of calculation</h2>

In [ ]:
data = data.loc[data['WS 120'] >= 3]
Data = np.array(data)

<h2>Get Alpha Values of Interest and extrapolate log law extrapolated wind speed</h2>

In [ ]:
alpha100_80 = np.divide(np.log(np.divide(Data[:, 3], Data[:, 2])), np.log(100/80))
alphaWS = np.multiply(Data[:, 3], (120/100)**alpha100_80)
pdAlpha = pd.DataFrame(alphaWS, columns=['Alpha WS'])

<h2>This block contains a majority of the data processing</h2>

In [ ]:
# Normalize current wind speed
ws = np.array(data.iloc[:, :5])
RegWS = pd.DataFrame(np.array(data.iloc[:, :5]))
RegWS.columns=['RegWS 40', 'RegWS 60', 'RegWS 80', 'RegWS 100', 'RegWS 120']
wsNorm = np.array(data['WS 100'])
WS = np.divide(ws, wsNorm[:, None])

# Put normalized vals back into array
Data[:, :5] = WS

# Normalize log law extrapolated wind speed
wsaReg = np.divide(alphaWS, wsNorm)
pdAlpha = pd.DataFrame(wsaReg, columns=['Alpha WS'])

# Normalize previous Wind Speed
prevws = np.array(data.iloc[:, 5:10])
prevwsNorm = np.array(data['Prev WS 100'])
prevWS = np.divide(prevws, prevwsNorm[:, None])
# Put normalized vals back into array
Data[:, 5:10] = prevWS

# Fix Directions to go -180º -> 180º
dirs = np.array(data.iloc[:, 20:25])
dirs[dirs>180] = dirs[dirs>180]-360

# Elevation Angle Data
angles = np.array(Angles)
dirData = np.array(data.iloc[:, 20:25])
dirData[dirData>180] = dirData[dirData>180]-360
elevAngle = np.zeros_like(dirData)
elevAngle[(dirData>0) & (dirData<=30)] = angles[0]
elevAngle[(dirData>30) & (dirData<=60)] = angles[1]
elevAngle[(dirData>60) & (dirData<=90)] = angles[2]
elevAngle[(dirData>90) & (dirData<=120)] = angles[3]
elevAngle[(dirData>120) & (dirData<=150)] = angles[4]
elevAngle[(dirData>150) & (dirData<=180)] = angles[5]
elevAngle[(dirData>-180) & (dirData<=-150)] = angles[6]
elevAngle[(dirData>-150) & (dirData<=-120)] = angles[7]
elevAngle[(dirData>-120) & (dirData<=-90)] = angles[8]
elevAngle[(dirData>-90) & (dirData<=-60)] = angles[9]
elevAngle[(dirData>-60) & (dirData<=-30)] = angles[10]
elevAngle[(dirData>-30) & (dirData<=0)] = angles[11]
Elev = pd.DataFrame(elevAngle[:, :4], columns=['Elev 40', 'Elev 60', 'Elev 80',
                                              'Elev 100'])

# Wind Shear Data - dudz
sep20m = np.array(data.iloc[:, :4])
grad = np.gradient(sep20m, 20, axis=1)
pdGrad = pd.DataFrame(grad, columns=['Shear 40m', 'Shear 60m', 'Shear 80m',
                                    'Shear 100'])

# Concatenate all data into a single Pandas dataframe
pandaData = pd.DataFrame(Data)
pandaData.columns = data.columns
pdData = pd.concat([pandaData, pdAlpha, pdGrad, Elev, RegWS], axis = 1, sort = False)
Cols = pdData.columns

<h2>Save Data</h2>

In [ ]:
pdData.to_csv('NondimData120.csv')